_Strategy:_

1. mongo dump / reload on a local machine

2. Compose ranges where start is run_start time and stop is run_stop time

3. Compose a dictionary keyed on run_start oid that holds run_start and run_stop tuples

4. iterate over the range dictionary and get distinct event_descriptor_ids for events that fall under each range and put these descriptors in a dictionary formatted as rs_desc_pairs['run_start_id'] = distinct_event_descriptors

5. By iterating over the dict's contents in step 4, findthe cases that an event_descriptor is associated with more than one run_start (there are 8 and this is due to creation of two consecutive run_starts not too far apart in time). Pick the run_start that is closest to the event_descriptor in time. This step results in desc_rstart_pairs dict where each event_descriptor has a corresponding run_start

6. For event_descriptors that I have in desc_rstart_pairs, get distinct data fields. This information will be stored in dt_key dict where keys are event_descriptor oids and contents are the corresponding data keys and their shapes 

7. Extract the source of all keys from the beamline. Get all keys from the beamline 

8. Using steps 5, 6, and 7 create and insert all event_descriptors



In [1]:
from pymongo import MongoClient
from bson import ObjectId
from pymongo.errors import OperationFailure, InvalidDocument
from collections import deque
from difflib import SequenceMatcher

In [2]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MIGRATION_DB = 'datastore2'
# 1) mongo dump / reload on a local machine
pymongo_client = MongoClient(MONGO_HOST, MONGO_PORT)
database = pymongo_client['datastore2']

In [3]:
source_keys = {}
source_keys['epu1_gap'] = 'XF:23ID-ID{EPU:1-Ax:Gap}Pos-SP'
source_keys['epu2_gap'] = 'XF:23ID-ID{EPU:2-Ax:Gap}Pos-SP'
source_keys['epu1_phase'] = 'XF:23ID-ID{EPU:1-Ax:Phase}Pos-SP'
source_keys['epu2_phase'] = 'XF:23ID-ID{EPU:2-Ax:Phase}Pos-SP'
source_keys['fe_xc'] = 'XF:23ID-ID{EPU:2-Ax:Phase}Pos-SP'
source_keys['fe_yc'] = 'FE:C23A-OP{Slt:12-Ax:Y}center'
source_keys['fe_xg'] = 'FE:C23A-OP{Slt:12-Ax:X}size'
source_keys['fe_yg'] = 'FE:C23A-OP{Slt:12-Ax:Y}size'
source_keys['delta'] = 'XF:23ID1-ES{Dif-Ax:Del}Mtr'
source_keys['gamma'] = 'XF:23ID1-ES{Dif-Ax:Gam}Mtr'
source_keys['theta'] = 'XF:23ID1-ES{Dif-Ax:Th}Mtr'
source_keys['th'] = 'XF:23ID1-ES{Dif-Ax:Th}Mtr'
source_keys['sx']='XF:23ID1-ES{Dif-Ax:X}Mtr'
source_keys['sy']='XF:23ID1-ES{Dif-Ax:SY}Pos-SP'
source_keys['sz']='XF:23ID1-ES{Dif-Ax:SZ}Pos-SP'
source_keys['say'] ='XF:23ID1-ES{Dif-Ax:Y}Mtr'
source_keys['saz'] = 'XF:23ID1-ES{Dif-Ax:Z}Mtr'
source_keys['cryoangle'] = 'XF:23ID1-ES{Dif-Cryo}Pos:Angle-SP'
source_keys['ptx']=  'XF:23ID1-ES{Dif:Lens-Ax:TopX}Mtr'
source_keys['npty'] = 'XF:23ID1-ES{Dif:Lens-Ax:TopY}Mtr'
source_keys['nptz'] = 'XF:23ID1-ES{Dif:Lens-Ax:TopZ}Mtr'
source_keys['npbx'] = 'XF:23ID1-ES{Dif:Lens-Ax:BtmX}Mtr'
source_keys['npby'] = 'XF:23ID1-ES{Dif:Lens-Ax:BtmY}Mtr'
source_keys['npbz'] = 'XF:23ID1-ES{Dif:Lens-Ax:BtmZ}Mtr'
source_keys['es_diag1_y'] = 'XF:23ID1-ES{Diag:1-Ax:Y}Mtr'
source_keys['eta'] = 'XF:23ID1-ES{Diag:1-Ax:Eta}Mtr'
source_keys['temp_sp'] = 'XF:23ID1-ES{TCtrl:1-Out:1}T-SP'
source_keys['sm_i'] = 'XF:23ID1-ES{K2611:1}Val:SP-I'
source_keys['sm_v'] = 'XF:23ID1-ES{K2611:1}Val:SP-E'
source_keys['sm_r'] = 'XF:23ID1-ES{K2611:1}Val:SP-R'    
source_keys['m1a_z'] = 'XF:23IDA-OP:1{Mir:1-Ax:Z}Mtr_POS_SP'
source_keys['m1a_y'] ='XF:23IDA-OP:1{Mir:1-Ax:Y}Mtr_POS_SP'
source_keys['m1a_x'] = 'XF:23IDA-OP:1{Mir:1-Ax:X}Mtr_POS_SP'
source_keys['m1a_pit'] = 'XF:23IDA-OP:1{Mir:1-Ax:Pit}Mtr_POS_SP'
source_keys['m1a_yaw'] = 'XF:23IDA-OP:1{Mir:1-Ax:Yaw}Mtr_POS_SP'
source_keys['m1a_rol'] = 'XF:23IDA-OP:1{Mir:1-Ax:Rol}Mtr_POS_SP'
source_keys['m1b1_z'] = 'XF:23IDA-OP:2{Mir:1A-Ax:Z}Mtr_POS_SP'
source_keys['m1b1_y'] = 'XF:23IDA-OP:2{Mir:1A-Ax:Y}Mtr_POS_SP'
source_keys['fccd'] = 'XF:23ID1-ES{FCCD}'
source_keys['fccd_acquire_time'] = 'XF:23ID1-ES{FCCD}:AcquireTime'
source_keys['fccd_acquire'] = 'XF:23ID1-ES{FCCD}:Acquire_RBV'
source_keys['fccd_acquire_period'] = 'XF:23ID1-ES{FCCD}:AcquirePeriod'
source_keys['fccd_acquire_capture'] = 'XF:23ID1-ES{FCCD}:Capture'
source_keys['fccd_num_images'] = 'XF:23ID1-ES{FCCD}:NumImages'
source_keys['fccd_num_captured'] = 'XF:23ID1-ES{FCCD}:NumCaptured'


source_keys['fccd_image'] = 'XF:23ID1-ES{FCCD}:Capture'
source_keys['fccd_image_mode'] = 'XF:23ID1-ES{FCCD}:ImageMode'
source_keys['fccd_capture'] = 'XF:23ID1-ES{FCCD}:Capture_RBV'
source_keys['fccd_file_name'] = 'XF:23ID1-ES{FCCD}:FileName'
source_keys['fccd_array_counter'] = 'XF:23ID1-ES{FCCD}:ArrayCounter'
source_keys['fccd_num_exposures'] = 'XF:23ID1-ES{FCCD}:NumExposures'
source_keys['fccd_file_template'] = 'XF:23ID1-ES{FCCD}:FileTemplate'
source_keys['fccd_arraysize0'] = 'XF:23ID1-ES{FCCD}:ArraySize0_RBV' 
source_keys['fccd_arraysize1'] = 'XF:23ID1-ES{FCCD}:ArraySize1_RBV'
source_keys['fccd_filepath_exists']= 'XF:23ID1-ES{FCCD}:FilePathExists_RBV'
source_keys['fccd_file_path'] = 'XF:23ID1-ES{FCCD}:FilePath'
for _ in range(1, 31):
    source_keys['fccd_stats_total'+str(_)] = 'XF:23ID1-ES{FCCD}:TOTAL_RBC.S' + str(_)
for _ in range(1, 10):
    source_keys['diag3_cam_stats_total'+str(_)] = 'XF:23ID1-BI{Diag:3-Cam:1}:TOTAL_RBCS.S' + str(_)
source_keys['m1b1_x'] = 'XF:23IDA-OP:2{Mir:1A-Ax:X}Mtr_POS_SP'
source_keys['m1b1_pit'] = 'XF:23IDA-OP:2{Mir:1A-Ax:Pit}Mtr_POS_SP'
source_keys['m1b1_yaw'] = 'XF:23IDA-OP:2{Mir:1A-Ax:Yaw}Mtr_POS_SP'
source_keys['m1b1_rol'] = 'XF:23IDA-OP:2{Mir:1A-Ax:Rol}Mtr_POS_SP'
source_keys['m1b2_z'] =   'XF:23IDA-OP:2{Mir:1B-Ax:Z}Mtr_POS_SP'
source_keys['m1b2_y'] =   'XF:23IDA-OP:2{Mir:1B-Ax:Y}Mtr_POS_SP'
source_keys['m1b2_x'] =   'XF:23IDA-OP:2{Mir:1B-Ax:X}Mtr_POS_SP'
source_keys['m1b2_pit'] = 'XF:23IDA-OP:2{Mir:1B-Ax:Pit}Mtr_POS_SP'
source_keys['m1b2_yaw'] = 'XF:23IDA-OP:2{Mir:1B-Ax:Yaw}Mtr_POS_SP'
source_keys['m1b2_rol'] = 'XF:23IDA-OP:2{Mir:1B-Ax:Rol}Mtr_POS_SP'
source_keys['pgm_energy'] = 'XF:23ID1-OP{Mono}Enrgy-SP'
source_keys['pgm_mir_pit'] = 'XF:23ID1-OP{Mono-Ax:MirP}Mtr'
source_keys['pgm_grt_pit'] = 'XF:23ID1-OP{Mono-Ax:GrtP}Mtr'
source_keys['pgm_mir_x'] = 'XF:23ID1-OP{Mono-Ax:MirX}Mtr' 
source_keys['pgm_grt_x'] = 'XF:23ID1-OP{Mono-Ax:GrtX}Mtr'
source_keys['pgm_energy_sp'] = 'XF:23ID1-OP{Mono}Enrgy-SP'
source_keys['pgm_energy_i'] = 'XF:23ID1-OP{Mono}Enrgy-I'
source_keys['m3a_x'] = 'XF:23ID1-OP{Mir:3-Ax:XAvg}Mtr'
source_keys['m3a_pit'] = 'XF:23ID1-OP{Mir:3-Ax:P}Mtr'
source_keys['m3a_bdr'] = 'XF:23ID1-OP{Mir:3-Ax:Bdr}Mtr'
source_keys['sh_y'] = 'XF:23ID1-OP{Sh:Fast-Ax:Y}Mtr'
source_keys['sh_x'] = 'XF:23ID1-OP{Sh:Fast-Ax:X}Mtr'
source_keys['slt1_xg'] = 'XF:23ID1-OP{Slt:1-Ax:XGap}Mtr'
source_keys['slt1_xc'] = 'XF:23ID1-OP{Slt:1-Ax:XCtr}Mtr'
source_keys['slt1_yg'] = 'XF:23ID1-OP{Slt:1-Ax:YGap}Mtr'
source_keys['slt1_yc'] = 'XF:23ID1-OP{Slt:1-Ax:YCtr}Mtr'
source_keys['slt2_xg'] = 'XF:23ID1-OP{Slt:2-Ax:XGap}Mtr'
source_keys['slt2_xc'] = 'XF:23ID1-OP{Slt:2-Ax:XCtr}Mtr'
source_keys['slt2_yg'] = 'XF:23ID1-OP{Slt:2-Ax:YGap}Mtr'
source_keys['slt2_yc'] = 'XF:23ID1-OP{Slt:2-Ax:YCtr}Mtr'
source_keys['slt3_x'] =  'XF:23ID1-OP{Slt:3-Ax:X}Mtr'
source_keys['slt3_y'] =  'XF:23ID1-OP{Slt:3-Ax:Y}Mtr'
source_keys['diag2_y'] = 'XF:23ID1-BI{Diag:2-Ax:Y}Mtr'
source_keys['diag3_y'] = 'XF:23ID1-BI{Diag:3-Ax:Y}Mtr'
source_keys['diag5_y'] = 'XF:23ID1-BI{Diag:5-Ax:Y}Mtr'
source_keys['diag6_y'] = 'XF:23ID1-BI{Diag:6-Ax:Y}Mtr'
source_keys['diag6_pid'] = 'XF:23ID1-OP{FBck}PID-SP'
source_keys['sclr'] = 'XF:23ID1-ES{Sclr:1}'
source_keys['sclr_time'] = 'XF:23ID1-ES{Sclr:1}.T'
source_keys['sclr_trig'] = 'XF:23ID1-ES{Sclr:1}.CNT'
source_keys['sclr_ch1'] = 'XF:23ID1-ES{Sclr:1}.S1'
source_keys['sclr_ch2'] = 'XF:23ID1-ES{Sclr:1}.S2'
source_keys['sclr_ch3'] = 'XF:23ID1-ES{Sclr:1}.S3'
source_keys['sclr_ch4'] = 'XF:23ID1-ES{Sclr:1}.S4'
source_keys['sclr_ch5'] = 'XF:23ID1-ES{Sclr:1}.S5'
source_keys['sclr_ch6'] = 'XF:23ID1-ES{Sclr:1}.S6'
source_keys['sclr_chan1'] = 'XF:23ID1-ES{Sclr:1}.S1'
source_keys['sclr_chan2'] = 'XF:23ID1-ES{Sclr:1}.S2'
source_keys['sclr_chan3'] = 'XF:23ID1-ES{Sclr:1}.S3'
source_keys['sclr_chan4'] = 'XF:23ID1-ES{Sclr:1}.S4'
source_keys['sclr_chan5'] = 'XF:23ID1-ES{Sclr:1}.S5'
source_keys['sclr_chan6'] = 'XF:23ID1-ES{Sclr:1}.S6'
source_keys['sclr_chan7'] = 'XF:23ID1-ES{Sclr:1}.S7'
source_keys['sclr_chan8'] = 'XF:23ID1-ES{Sclr:1}.S8'
source_keys['sclr_chan9'] = 'XF:23ID1-ES{Sclr:1}.S9'
source_keys['sclr_chan10'] = 'XF:23ID1-ES{Sclr:1}.S10'
source_keys['sclr_chan11'] = 'XF:23ID1-ES{Sclr:1}.S11'

for _ in range(1, 36):
    source_keys['sclr_chan'+str(_)] = 'XF:23ID1-ES{Sclr:1}.S' + str(_)
source_keys['nptx'] = 'XF:23ID1-ES{Dif:Lens-Ax:TopX}Mtr'
source_keys['npty'] = 'XF:23ID1-ES{Dif:Lens-Ax:TopY}Mtr'
source_keys['ring_curr'] = 'XF:23ID-SR{}I-I'
source_keys['temp_a'] = 'XF:23ID1-ES{TCtrl:1-Chan:A}T-I'
source_keys['temp_b'] = 'XF:23ID1-ES{TCtrl:1-Chan:B}T-I'
source_keys['fs1_cam'] ='XF:23IDA-BI:1{FS:1-Cam:1}'
source_keys['diag3_cam'] = 'XF:23ID1-BI{Diag:3-Cam:1}'
source_keys['diag5_cam'] = 'XF:23ID1-BI{Diag:5-Cam:1}'
source_keys['diag6_cam'] = 'XF:23ID1-BI{Diag:6-Cam:1}'
source_keys['dif_beam_cam'] = 'XF:23ID1-ES{Dif-Cam:Beam}'
source_keys['fs1_cam'] = 'XF:23IDA-BI:1{FS:1-Cam:1}'
source_keys['diag3_cam'] = 'XF:23ID1-BI{Diag:3-Cam:1}'
source_keys['diag5_cam'] = 'XF:23ID1-BI{Diag:5-Cam:1}'

source_keys['diag6_cam'] = 'XF:23ID1-BI{Diag:6-Cam:1}'
source_keys['diag6_cam_image_mode'] = 'XF:23ID1-BI{Diag:6-Cam:1}:ImageMode'
source_keys['diag6_cam_acquire_time'] = 'XF:23ID1-BI{Diag:6-Cam:1}:AcquireTime'
source_keys['diag6_cam_acquire_period'] = 'XF:23ID1-BI{Diag:6-Cam:1}:AcquirePeriod'
source_keys['diag6_cam_stats_total1'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats1:Total_RBV'
source_keys['diag6_cam_stats_total2'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats2:Total_RBV'
source_keys['diag6_cam_stats_total3'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats3:Total_RBV'
source_keys['diag6_cam_stats_total4'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats4:Total_RBV'
source_keys['diag6_cam_stats_total5'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats5:Total_RBV'
source_keys['diag6_cam_array_counter'] = 'XF:23ID1-BI{Diag:6-Cam:1}:ArrayCounter'
source_keys['diag6_cam_num_images'] = 'XF:23ID1-BI{Diag:6-Cam:1}:NumImages'
source_keys['diag6_cam_num_exposures'] = 'XF:23ID1-BI{Diag:6-Cam:1}:NumExposures'
source_keys['diag6_cam_acquire'] = 'XF:23ID1-BI{Diag:6-Cam:1}:Acquire'

source_keys['diag5_cam_image_mode'] = 'XF:23ID1-BI{Diag:5-Cam:1}:ImageMode'
source_keys['diag5_cam_stats_total1'] = 'XF:23ID1-BI{Diag:5-Cam:1}Stats1:Total_RBV'
source_keys['diag5_cam_stats_total2'] = 'XF:23ID1-BI{Diag:5-Cam:1}Stats2:Total_RBV'
source_keys['diag5_cam_stats_total3'] = 'XF:23ID1-BI{Diag:5-Cam:1}Stats3:Total_RBV'
source_keys['diag5_cam_stats_total4'] = 'XF:23ID1-BI{Diag:5-Cam:1}Stats4:Total_RBV'
source_keys['diag5_cam_stats_total5'] = 'XF:23ID1-BI{Diag:5-Cam:1}Stats5:Total_RBV'
source_keys['diag5_cam_acquire_time'] = 'XF:23ID1-BI{Diag:5-Cam:1}:AcquireTime'
source_keys['diag5_cam_acquire_period'] = 'XF:23ID1-BI{Diag:5-Cam:1}:AcquirePeriod'
source_keys['diag5_cam_array_counter'] = 'XF:23ID1-BI{Diag:5-Cam:1}:ArrayCounter'
source_keys['diag5_cam_num_images'] = 'XF:23ID1-BI{Diag:5-Cam:1}:NumImages'
source_keys['diag5_cam_acquire'] = 'XF:23ID1-BI{Diag:5-Cam:1}:Acquire'
source_keys['diag5_cam_num_exposures'] = 'XF:23ID1-BI{Diag:5-Cam:1}:NumExposures'

source_keys['diag3_cam_image_mode'] = 'XF:23ID1-BI{Diag:3-Cam:1}:ImageMode'
source_keys['diag3_cam'] = 'XF:23ID1-BI{Diag:3-Cam:1}'
source_keys['diag3_cam_acquire_time'] = 'XF:23ID1-BI{Diag:3-Cam:1}:AcquireTime'
source_keys['diag3_cam_acquire'] = 'XF:23ID1-BI{Diag:3-Cam:1}:Acquire'
source_keys['diag3_cam_acquire_period'] = 'XF:23ID1-BI{Diag:3-Cam:1}:AcquirePeriod'
source_keys['diag3_cam_stats_total1'] = 'XF:23ID1-BI{Diag:3-Cam:1}Stats1:Total_RBV'
source_keys['diag3_cam_stats_total2'] = 'XF:23ID1-BI{Diag:3-Cam:1}Stats2:Total_RBV'
source_keys['diag3_cam_stats_total3'] = 'XF:23ID1-BI{Diag:3-Cam:1}Stats3:Total_RBV'
source_keys['diag3_cam_stats_total4'] = 'XF:23ID1-BI{Diag:3-Cam:1}Stats4:Total_RBV'
source_keys['diag3_cam_stats_total5'] = 'XF:23ID1-BI{Diag:3-Cam:1}Stats5:Total_RBV'
source_keys['diag3_cam_array_counter'] = 'XF:23ID1-BI{Diag:3-Cam:1}:ArrayCounter'
source_keys['diag3_cam_num_images'] = 'XF:23ID1-BI{Diag:3-Cam:1}:NumImages'
source_keys['diag3_cam_num_exposures'] = 'XF:23ID1-BI{Diag:3-Cam:1}:NumExposures'





source_keys['dif_beam_cam'] = 'XF:23ID1-ES{Dif-Cam:Beam}'
source_keys['dif_beam_cam_stats_total1'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats1:Total_RBV'
source_keys['dif_beam_cam_stats_total2'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats2:Total_RBV'
source_keys['dif_beam_cam_stats_total3'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats3:Total_RBV'
source_keys['dif_beam_cam_stats_total4'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats4:Total_RBV'
source_keys['dif_beam_cam_stats_total5'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats5:Total_RBV'

source_keys['dif_beam_cam'] = 'XF:23ID1-ES{Dif-Cam:Beam}'
source_keys['dif_beam_cam_num_images'] = 'XF:23ID1-ES{Dif-Cam:Beam}:NumImages'
source_keys['dif_beam_cam_acquire_time'] = 'XF:23ID1-ES{Dif-Cam:Beam}:AcquireTime'
source_keys['dif_beam_cam_acquire_period'] = 'XF:23ID1-ES{Dif-Cam:Beam}:AcquirePeriod'


source_keys['dif_beam_cam_stats_total1'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats1:Total_RBV'
source_keys['dif_beam_cam_stats_total2'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats2:Total_RBV'
source_keys['dif_beam_cam_stats_total3'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats3:Total_RBV'
source_keys['dif_beam_cam_stats_total4'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats4:Total_RBV'
source_keys['dif_beam_cam_stats_total5'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats5:Total_RBV'


source_keys['None'] = 'XF:23ID1-ES{Dif-Cam:Beam}'
source_keys['None_acquire_time'] = 'XF:23ID1-ES{Dif-Cam:Beam}:AcquireTime'
source_keys['None_num_images'] = 'XF:23ID1-ES{Dif-Cam:Beam}:NumImages'


source_keys['None_acquire_period'] = 'XF:23ID1-ES{Dif-Cam:Beam}:AcquirePeriod'
source_keys['None_image'] = 'XF:23ID1-ES{Dif-Cam:Beam}:Capture_RBV'
source_keys['None_stats_total1'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats1:Total_RBV'
source_keys['None_stats_total2'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats2:Total_RBV'
source_keys['None_stats_total3'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats3:Total_RBV'
source_keys['None_stats_total4'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats4:Total_RBV'
source_keys['None_stats_total5'] = 'XF:23ID1-ES{Dif-Cam:Beam}Stats5:Total_RBV'



source_keys['ccdtest'] = 'XF:23ID1-ES{Tst-Cam:1}'
source_keys['ccdtest_image'] = 'XF:23ID1-ES{Tst-Cam:1}:Capture_RBV'
source_keys['ccdtest_acquire_time'] = 'XF:23ID1-ES{Tst-Cam:1}:AcquireTime'
source_keys['ccdtest_acquire_period'] = 'XF:23ID1-ES{Tst-Cam:1}:AcquirePeriod'
source_keys['ccdtest_stats_total1'] = 'XF:23ID1-ES{Tst-Cam:1}Stats1:Total_RBV'
source_keys['ccdtest_stats_total2'] = 'XF:23ID1-ES{Tst-Cam:1}Stats2:Total_RBV'
source_keys['ccdtest_stats_total3'] = 'XF:23ID1-ES{Tst-Cam:1}Stats3:Total_RBV'
source_keys['ccdtest_stats_total4'] = 'XF:23ID1-ES{Tst-Cam:1}Stats4:Total_RBV'
source_keys['ccdtest_stats_total5'] = 'XF:23ID1-ES{Tst-Cam:1}Stats5:Total_RBV'
source_keys['ccdtest_array_counter'] = 'XF:23ID1-ES{Tst-Cam:1}:ArrayCounter'
source_keys['ccdtest_num_images'] = 'XF:23ID1-ES{Tst-Cam:1}:NumImages'


source_keys['pimte'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}'                                       
source_keys['pimte_num_images'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}:NumImages'                                       
source_keys['pimte_image'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}:Capture'   
source_keys['pimte_acquire_time'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}:AcquireTime'
source_keys['pimte_acquire_period'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}:AcquirePeriod'

source_keys['fshutter'] = 'XF:23ID1-TS{EVR:1-Out:FP0}Src:Scale-RB'
source_keys['topoff_inj'] = 'XF:23ID1-SR{TO-Inj}'
source_keys['topoff_btr'] = 'XF:23ID1-SR{TO-BS}'
source_keys['fccd_time'] = 'XF:23ID1-ES{FCCD-TS}'
source_keys['diag6_flyer1'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats1:'
source_keys['diag6_flyer5'] = 'XF:23ID1-BI{Diag:6-Cam:1}Stats5:'
source_keys['fccd_flyer5'] = 'XF:23ID1-ES{FCCD}Stats5:'
source_keys['pimte_cam'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}cam1:Amecquire_RBV'
source_keys['pimte_tot1'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats1:Total_RBV'
source_keys['pimte_tot2'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats2:Total_RBV'
source_keys['pimte_tot3'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats3:Total_RBV'
source_keys['pimte_tot4'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats4:Total_RBV'
source_keys['pimte_tot5'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats5:Total_RBV'
source_keys['pimte_stats_total1'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats1:Total_RBV'
source_keys['pimte_stats_total2'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats2:Total_RBV'
source_keys['pimte_stats_total3'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats3:Total_RBV'
source_keys['pimte_stats_total4'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats4:Total_RBV'
source_keys['pimte_stats_total5'] = 'XF:23ID1-ES{Dif-Cam:PIMTE}Stats5:Total_RBV'
source_keys['XF:23ID1-OP{Mono}Enrgy-I'] = 'XF:23ID1-OP{Mono}Enrgy-I'
source_keys['XF:23ID1-OP{Mono}Enrgy-RB'] = 'XF:23ID1-OP{Mono}Enrgy-RB'
source_keys['xf[dot]23id1-es{dif-cam[dot]beam}cam1[dot]stats2[dot]total_rbv']= 'xf.23id1-es{dif-cam.beam}cam1.stats2.total_rbv'
source_keys['XF:23ID1-OP{Mono}Enrgy-SP'] = 'XF:23ID1-OP{Mono}Enrgy-SP'
source_keys['fccd_image_lightfield'] = '' #?? 
source_keys['fccd_image_darkfield'] = '' #?? 

# unknown_source = []
# for i in unique_keys:
#     try:
#         source_keys[i]
#     except KeyError:
#         unknown_source.append(i)
# print(len(unknown_source))
# for u in unknown_source:
#     print(u)





In [4]:
desc_oids = list()
desc_oids = database.event.distinct('descriptor_id')
print(len(desc_oids))

13801


In [5]:
# 2. Compose ranges where start is run_start time and stop is run_stop time

# 3. Compose a dictionary keyed on run_start oid that holds run_start and run_stop tuples

rstt_crsr = database.run_start.find()
pairs = dict()
for rstart in rstt_crsr:
    try:
        run_stop = next(database.run_stop.find({'run_start_id': rstart['_id']}))
    except StopIteration:
        run_stop = None 
    # there are some rstop that are not created. What to do with these!? Igonore for now
    if run_stop:
        time_range = (rstart['time'], run_stop['time'])
        pairs[rstart['_id']] = time_range

In [6]:
# 4. iterate over the range dictionary and get distinct event_descriptor_ids for events that fall under each range and put these descriptors in a dictionary formatted as rs_desc_pairs['run_start_id'] = distinct_event_descriptors

rs_desc_pairs = {}
for k, v in pairs.items():
    # give me all the distinct event descriptors for events in the
    # time range between start and stop
    query = {'time': {'$gt': v[0],'$lt': v[1]}}
    rs_desc_pairs[k] = database.event.find(query).distinct(key='descriptor_id')
    # rs_desc_pairs holds run_start event_descriptor pairs!

In [7]:
# 5. By iterating over the dict's contents in step 4, find 
# the cases that an event_descriptor is associated with more than 
# one run_start (there are 8 and this is due to creation of two consecutive 
# run_starts not too far apart in time). Pick the run_start that is closest 
# to the event_descriptor in time. This step results in desc_rstart_pairs dict where each event_descriptor has a corresponding run_start


descs = deque()
desc_rstart_pairs = {}
def similar(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()
# One run_start can have multiple descriptors. rs_desc_pairs holds run_start (k) and its
# corresponding descriptors (v)
for k,v in rs_desc_pairs.items():
    for _ in list(v):
        # Check if this descriptor has already been seen
        # if so, compare the two run_starts and pick the one closest
        # to the descriptor in time
        if _ not in descs:
            descs.append(_)
            desc_rstart_pairs[_] = k
        else:
            # For the 8 overlapping descriptors, select the most likely
            # run_start (oid is +1 is as close as it can possibly get)
            print('run_start 1', desc_rstart_pairs[_].generation_time,
                  'run_start 2', k.generation_time,
                  'descriptor', _.generation_time)    
            if similar(desc_rstart_pairs[_], _) < similar(k, _):
                desc_rstart_pairs[_] = k

run_start 1 2015-06-19 00:52:58+00:00 run_start 2 2015-06-19 00:48:30+00:00 descriptor 2015-06-19 00:53:01+00:00
run_start 1 2015-06-19 00:50:51+00:00 run_start 2 2015-06-19 00:48:30+00:00 descriptor 2015-06-19 00:50:54+00:00
run_start 1 2015-02-19 20:51:47+00:00 run_start 2 2015-02-19 20:51:47+00:00 descriptor 2015-02-19 20:51:47+00:00
run_start 1 2015-02-19 20:51:47+00:00 run_start 2 2015-02-19 20:51:47+00:00 descriptor 2015-02-19 20:51:47+00:00
run_start 1 2015-06-19 00:48:30+00:00 run_start 2 2015-06-19 00:53:54+00:00 descriptor 2015-06-19 00:53:57+00:00
run_start 1 2015-06-19 00:48:30+00:00 run_start 2 2015-06-19 00:53:54+00:00 descriptor 2015-06-19 00:48:39+00:00
run_start 1 2015-06-18 21:16:27+00:00 run_start 2 2015-06-18 21:15:21+00:00 descriptor 2015-06-18 21:15:25+00:00
run_start 1 2015-06-18 21:16:27+00:00 run_start 2 2015-06-18 21:15:21+00:00 descriptor 2015-06-18 21:16:30+00:00


In [8]:
# 6. For event_descriptors that I have in desc_rstart_pairs, get distinct data fields. 
# This information will be stored in dt_key dict where keys are event_descriptor oids and contents are the corresponding data keys and their shapes 
data_key_templates = {}
leftovers = []
unique_keys = []
for d in descs:
    dt_key = {}
    try:
        res = database.event.find({"descriptor_id" : ObjectId(d)}).distinct(key='data')[0]
    except OperationFailure:
        leftovers.append(d)
    for k, v in res.items():
        # For the timebeing!!!!
        try:
            source = source_keys[k]
        except KeyError:
            source = ''
        
        if v[0] is not None:
            if (type(v[0])==int) or (type(v[0])==float):
                data_type = 'number'
            elif (type(v[0])==str):
                data_type = 'array'
            else:
                data_type = type(v[0])
        else:
            data_type = None
        try:
            if (1356998400.0 <= v[1] <= 1452612569587.0): #2013 to now
                shape = []
            else:
                shape = (len(v),)
        except TypeError:
            shape = []
        if data_type == 'array':
            dt_key[k] = {'shape': shape, 'dtype': data_type, 
                         'source': source,'external': 'FILESTORE:'}
        else:
            dt_key[k] = {'shape': shape, 'dtype': data_type, 'source': source}
    data_key_templates[d] = dt_key

# the difference in the number of different descriptor count is
# due to run_stop-less run_starts
# There is a 16 MB limit on distinct. SOmetimes, even the smallest documents 
# get caught in the threshold and get neglected. Catch those that are excluded and 

In [10]:
# Add the leftovers to the dict
for l in leftovers:
    dt_key = {}
    res = database.event.find({"descriptor_id" : ObjectId(l)})
    data = next(res)['data']
    for k, v in data.items():
#         if k not in unique_keys:
#             unique_keys.append(k)
        try:
            source = source_keys[k]
        except KeyError:
            source = ''
        if v[0] is not None:
            if (type(v[0])==int) or (type(v[0])==float):
                data_type = 'number'
            elif (type(v[0])==str):
                data_type = 'array'
            else:
                data_type = type(v[0])
        else:
            data_type = None

        try:
            if (1356998400.0 <= v[1] <= 1452612569587.0): #2013 to now
                shape = []
            else:
                shape = (len(v),)
        except TypeError:
            shape = []
        if data_type == 'array':
            dt_key[k] = {'shape': shape, 'dtype': data_type, 'source': source, 
                         'external': 'FILESTORE:'}
        else:
            dt_key[k] = {'shape': shape, 'dtype': data_type, 'source': source}
        data_key_templates[l] = dt_key

In [11]:
# sanity check. Make sure events are between stop and start for "some" ObjectId
t = ObjectId('54e5f9b87368e36ad949a25b')
print('RunStart time', next(database.run_start.find({'_id': t}))['time'])
print(rs_desc_pairs[t][0])
crsr = database.event.find({'descriptor_id': rs_desc_pairs[t][0]})
for c in crsr:
    print('Event time ',c['time'], c['_id'])
print('RunStop time', next(database.run_stop.find({'run_start_id': t}))['time'])

RunStart time 1424357816.968672
54e5f9da7368e36ad949a25c
Event time  1424357850.284224 54e5f9da7368e36ad949a25d
Event time  1424357883.543736 54e5f9fb7368e36ad949a25e
Event time  1424357913.724856 54e5fa197368e36ad949a25f
Event time  1424357946.947482 54e5fa3a7368e36ad949a260
Event time  1424357973.540224 54e5fa557368e36ad949a261
Event time  1424358006.766089 54e5fa767368e36ad949a262
Event time  1424358033.844119 54e5fa917368e36ad949a263
Event time  1424358067.188064 54e5fab37368e36ad949a264
Event time  1424358093.697332 54e5facd7368e36ad949a265
Event time  1424358126.950872 54e5faee7368e36ad949a266
Event time  1424358153.969373 54e5fb0a7368e36ad949a267
RunStop time 1424358154.394573


In [12]:
a = [str(key).split(', ') for key in data_key_templates.keys()]
print(a[1199][0])
data_key_templates[ObjectId(a[199][0])]

56418f487368e3fcecde0eca


{'delta': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Ax:Del}Mtr'},
 'dif_beam_cam_acquire_period': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}:AcquirePeriod'},
 'dif_beam_cam_acquire_time': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}:AcquireTime'},
 'dif_beam_cam_stats_total1': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}Stats1:Total_RBV'},
 'dif_beam_cam_stats_total2': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}Stats2:Total_RBV'},
 'dif_beam_cam_stats_total3': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}Stats3:Total_RBV'},
 'dif_beam_cam_stats_total4': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}Stats4:Total_RBV'},
 'dif_beam_cam_stats_total5': {'dtype': 'number',
  'shape': [],
  'source': 'XF:23ID1-ES{Dif-Cam:Beam}Stats5:Total_RBV'},
 'epu1_gap': {'dtype': 'number',
  'shape': [],
  'sour

In [ ]:
# 8. Using steps 5, 6, and 7 create and insert all event_descriptors
import uuid
descriptor_dict = {}
for d in descs:
    rs = next(database.run_start.find({'_id': desc_rstart_pairs[d]}))
    
    descriptor_dict[d] = {'_id': d,
                          'run_start_id': desc_rstart_pairs[d], 
                          'data_keys': data_key_templates[d],
                          'time':rs['time']+1,
                          'uid': str(uuid.uuid4())}
    try:
        database.event_descriptor.insert(descriptor_dict[d])
    except InvalidDocument:
        print(descriptor_dict[d])

In [ ]:
from databroker import DataBroker, get_events
db = DataBroker[-10]
k = get_events(db)

for l in range(10):
    try:
        print(next(k))
    except StopIteration:
        print('done')
